1._________________ Setup and load the data___________________________

1.1 Install the dependencies

In [ ]:
!pip install tensorflow opencv-python matplotlib scikit-learn

In [1]:
import tensorflow as tf 
import os  
from tensorflow import keras
import cv2 
import imghdr 
from matplotlib import pyplot as plt 
import numpy as np
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.models import load_model

1.2 Remove dodgy images

In [ ]:
data_dir = 'data'
image_exts = ['jpeg','jpg', 'png']
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image) 
        try: 
            img = cv2.imread(image_path) 
            tip = imghdr.what(image_path)  
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path) 
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
          

1.3 Load the data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data',image_size=(128, 128))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

2. Pre-processing Data

2.1 Scale the data

In [ ]:
data = data.map(lambda x,y: (x/255, y))
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

2.2 Split the data

In [ ]:
train_size = int(len(data)*.75)
val_size = int(len(data)*0.2)
test_size = int(len(data)*0.05)
train = data.take(train_size) 
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

3. DEEP MODEL

3.1 Built deep learning model

3.1.1 CNN basic model

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), 1, activation='relu', input_shape=(128,128,3))) 
model.add(MaxPooling2D(2,2)) 

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten()) 

model.add(Dense(64, activation='relu'))
model.add(Dense(6,activation='softmax'))

3.1.2 CNN basic model regularized

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), 1, activation='relu', input_shape=(128,128,3))) 
model.add(MaxPooling2D(2,2)) 

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten()) 

model.add(Dense(64, activation='relu',kernel_regularizer = tf.keras.regularizers.l2(0.05)))
model.add(Dense(6,activation='softmax'))

3.1.3 CNN basic model regularized Fine-tuning (this part must be run after the fist training of the model 3.1.2)

In [ ]:
fine_tune_at = 4  
for layer in model.layers[:fine_tune_at]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

3.1.4 Alternative model (Pre-trained model VGG16)

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, 
                                         input_shape=(128, 128, 3))
base_model.trainable = True
fine_tune_at = 100  
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

3.2 Train

In [ ]:
model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(train, epochs=15, validation_data=val, callbacks=[tensorboard_callback])
model.summary()
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=5, validation_data=val, callbacks=[tensorboard_callback])

3.3 Plot performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

4. Evaluate Performance

4.1 Evaluate

In [ ]:
pre = Precision()
re = Recall()
acc = Accuracy()
y1=[]
y_true =[]
y_pred = [] 
l=0
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    l=l+1
    if l==test_size:
        for i in range(32):
            y1.append(yhat[i].argmax())
            y_true.append(y[i])
            y_pred.append(yhat[i].argmax())
    else:
        for i in range(32):
            y1.append(yhat[i].argmax())
            y_true.append(y[i])
            y_pred.append(yhat[i].argmax())
            
    pre.update_state(y, y1)
    re.update_state(y, y1)
    acc.update_state(y, y1)
    y1=[]

In [ ]:
print(f"presition: {pre.result().numpy()}, recall: {re.result().numpy()}, accuracy: {acc.result().numpy()}")

4.1.1 Confution Matrix

In [ ]:
cm = confusion_matrix(np.transpose(y_true), np.transpose(y_pred))
classes = ['carton', 'vidrio', 'plastico', 'papel', 'metal', 'basura comun']
def plot_confusion_matrix(cm, classes):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta Verdadera')
    plt.xlabel('Predicción')

# Graficar la matriz de confusión
plot_confusion_matrix(cm, classes)
plt.show()

4.2 Test

In [ ]:
img = cv2.imread('carton.jpg')
plt.imshow(img)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()
resize = tf.image.resize(img, (128,128))
plt.imshow(resize.numpy().astype(int))
plt.show()
yhat = model.predict(np.expand_dims(resize/255, 0))
yhat.argmax()

5. Save Model

In [ ]:
model.save(os.path.join('models','ImageTrashClassification.h5'))
new_model = load_model(os.path.join('models','ImageTrashClassification.h5'))
yhat=new_model.predict(np.expand_dims(resize/255, 0))
yhat.argmax()